In [ ]:
!pip install --upgrade stop_words  --quiet
!pip install clean-text --quiet
!python -m spacy download ru_core_news_lg --quiet
!pip install pymorphy2 --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.6 MB/s eta 0:00:00
2023-05-25 20:35:54.924137: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 20:35:56.384831: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 50.1 MB/s eta 0:00:00
✔ Download and installation successful
Y

In [ ]:
import stop_words as sw
import pandas as pd
import numpy as np
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
import pickle
from cleantext import clean
import gensim
import gensim.corpora as corpora
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
import spacy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import os
from os import listdir
import re
from os.path import isfile, join

In [ ]:
# предобработка
# Garbage cleaning
def clean_text(input):

  return clean(input,
      no_emoji=True,
      fix_unicode=True,               # fix various unicode errors
      to_ascii=False,                  # transliterate to closest ASCII representation
      lower=True,                     # lowercase text
      no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
      no_urls=True,                  # replace all URLs with a special token
      no_emails=True,                # replace all email addresses with a special token
      no_phone_numbers=True,         # replace all phone numbers with a special token
      no_numbers=True,               # replace all numbers with a special token
      no_digits=True,                # replace all digits with a special token
      no_currency_symbols=True,      # replace all currency symbols with a special token
      no_punct=True,                 # remove punctuations
      replace_with_punct="",          # instead of removing punctuations you may replace them
      replace_with_url="",
      replace_with_email="",
      replace_with_phone_number="",
      replace_with_number="",
      replace_with_digit="",
      replace_with_currency_symbol="",
      lang="en"                       # set to 'de' for German special handling
  )

def tokenize(cleaned_text):
  return cleaned_text.apply(word_tokenize)

# лемматизация
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
  texts_out = []
  nlp = spacy.load('ru_core_news_lg', disable=['parser', 'ner'])
  for sent in texts:
      doc = nlp(" ".join(sent)) 
      texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
  return texts_out

# удаление стоп слов
def remove_stopwords(sentence):
  stop_words = sw.get_stop_words('russian')
  return [word for word in sentence if word not in stop_words]

In [ ]:
def preprocess(df):
  cleaned_text = df.raw_text.apply(clean_text)
  tokenized_texts = tokenize(cleaned_text)
  lemmatized_texts_spacy = lemmatization(tokenized_texts.values)
  lemmatized_texts_spacy = pd.Series(lemmatized_texts_spacy, index=tokenized_texts.index)
  texts_wo_sw = lemmatized_texts_spacy.apply(remove_stopwords)
  comments=pd.Series(texts_wo_sw,index=tokenized_texts.index)
  df = df.assign(text = comments, length=comments.apply(len))
  return df

In [ ]:
# cleaned_text = tg_df.text.apply(clean_text)
# tokenized_texts = tokenize(cleaned_text)
# tokenized_texts_spacy = lemmatization(tokenized_texts.values)
# tokenized_texts_spacy = pd.Series(tokenized_texts_spacy, index=tokenized_texts.index)
# texts_wo_sw = tokenized_texts_spacy.apply(remove_stopwords)
# tg_comments = pd.Series(texts_wo_sw,index=tokenized_texts.index)

pymorphy2

In [ ]:
# import pymorphy2
# morph = pymorphy2.MorphAnalyzer()
# def lemmatize(doc):
#     tokens = []
#     for token in doc:
#         token = token.strip()
#         token = morph.normal_forms(token)[0]
#         tokens.append(token)
#     if len(tokens) > 2:
#         return 
# tokenized_texts.head(1).apply(lemmatize)[0]

# TG

In [ ]:
cwd = os.getcwd()
onlyfiles = [os.path.join(cwd, f) for f in os.listdir(cwd) if 
os.path.isfile(os.path.join(cwd, f))]
onlyfiles

['/content/zwsmos.csv',
 '/content/osoznannoyepotrebleniye.csv',
 '/content/my_green_bag_messages.csv',
 '/content/tg_ecochatik_messages.csv',
 '/content/tg_sobirator_messages.csv']

In [ ]:
full_df_tg = pd.DataFrame()
for file in onlyfiles:
  source = re.search(r'\/([\w.]+)\.csv$', file).group(1)
  df = pd.read_csv(file, index_col=0).assign(source=source).reset_index(drop=True)
  df = df.rename(columns={'0':'raw_text'})
  df = df.iloc[df.raw_text.drop_duplicates().index]
  df.text = df.raw_text.astype(str)
  full_df_tg = pd.concat([full_df_tg, df])
  print(source,df.shape)

zwsmos (388, 2)
osoznannoyepotrebleniye (857, 2)
my_green_bag_messages (411, 2)
tg_ecochatik_messages (10777, 2)
tg_sobirator_messages (490, 2)


In [ ]:
# tg_df = pd.concat([
#     pd.read_csv('/content/tg_ecochatik_messages.csv', index_col=0).assign(source='tg_ecochatic'),
#     pd.read_csv('/content/tg_sobirator_messages.csv', index_col=0).assign(source='tg_sobirator')
# ])
# tg_df = tg_df.rename(columns={'0':'raw_text'})
# tg_df.text = tg_df.raw_text.astype(str)
# tg_df.shape

(12420, 2)

In [ ]:
full_df_tg = preprocess(full_df_tg)
# tg_df = tg_df.assign(text = tg_comments, length=tg_comments.apply(len))

In [ ]:
full_df_tg.head()

,raw_text,source,text,length
0,Всем привет! \n\nУ нас новости. Мы приняли реш...,zwsmos,"[привет, новость, принять, решение, закрыть, с...",45
1,Мы хотим напомнить об основных правилах хранен...,zwsmos,"[напомнить, основный, правило, хранение, проду...",85
2,NaN,zwsmos,[],0
3,NaN,zwsmos,[],0
4,NaN,zwsmos,[],0


In [ ]:
full_df_tg.shape

(12923, 4)

In [ ]:
full_df_tg.to_csv('full_df_tg.csv')

# VK

In [ ]:
# df = pd.concat([
#     pd.read_csv('/content/vk_ed_zbs_messages.csv', index_col=0).assign(source='vk_ed_zbs'),
#     pd.read_csv('/content/vk_rsbor_messages.csv', index_col=0).assign(source='vk_rsbor')
# ])
# df = df.rename(columns={'0':'raw_text'})
# df.text = df.raw_text.astype(str)
# vk_df = df.copy()
# vk_df.shape

(7960, 2)

In [ ]:
import os
from os import listdir
import re
from os.path import isfile, join

cwd = os.getcwd()
onlyfiles = [os.path.join(cwd, f) for f in os.listdir(cwd) if 
os.path.isfile(os.path.join(cwd, f))]

In [ ]:
onlyfiles

['/content/vsemvermi_messages.csv',
 '/content/pererabotkinskaya_messages.csv',
 '/content/zerowasterus_messages.csv',
 '/content/zero_waste_style_messages.csv',
 '/content/eco_sfera_messages.csv',
 '/content/sobirator_messages.csv',
 '/content/zerowasteshop.moscow_messages.csv',
 '/content/vk_ed_zbs_messages.csv',
 '/content/fruktowcaeco_messages.csv',
 '/content/zero_waste_shop_messages.csv',
 '/content/vk_rsbor_messages.csv',
 '/content/greencospb_messages.csv',
 '/content/zerowaste_ru_messages.csv']

In [ ]:
full_df = pd.DataFrame()
for file in onlyfiles:
  source = re.search(r'\/([\w.]+)\.csv$', file).group(1)
  df = pd.read_csv(file, index_col=0).assign(source=source).reset_index(drop=True)
  df = df.rename(columns={'0':'raw_text'})
  df = df.iloc[df.raw_text.drop_duplicates().index]
  df.text = df.raw_text.astype(str)
  full_df = pd.concat([full_df, df])
  print(source,df.shape)


vsemvermi_messages (1559, 2)
pererabotkinskaya_messages (890, 2)
zerowasterus_messages (5357, 2)
zero_waste_style_messages (505, 2)
eco_sfera_messages (2975, 2)
sobirator_messages (2965, 2)
zerowasteshop.moscow_messages (738, 2)
vk_ed_zbs_messages (2366, 2)
fruktowcaeco_messages (252, 2)
zero_waste_shop_messages (2674, 2)
vk_rsbor_messages (4846, 2)
greencospb_messages (6333, 2)
zerowaste_ru_messages (15997, 2)


In [ ]:
full_df = preprocess(full_df)
full_df.to_csv('full_df_vk.csv')

In [ ]:
# vk_df = preprocess(vk_df)
# vk_df.to_csv('vk_df.csv')

# Youtube

In [ ]:
yt = pd.read_excel('/content/yt_comments_full.xlsx',index_col=0).reset_index(drop=True)
yt = yt.rename(columns={'textDisplay':'raw_text'})
yt = yt.iloc[yt.raw_text.drop_duplicates().index]
yt_df = yt[['raw_text', 'videoId']]
# yt.head()
yt.shape

(278781, 10)

In [ ]:
yt_df = preprocess(yt_df)

In [ ]:
yt_df.head()

,raw_text,videoId,text,length
0,Всем добра! Автору успехов),Q6xX28WaNRc,"[добро, автор, успех]",3
1,"Респект, идёшь в нужном направлении!",Q6xX28WaNRc,"[респект, идёшь, нужный, направление]",4
2,"Что значит &quot;поел, попил, поспал..&quot;? ...",Q6xX28WaNRc,"[значить, поесть, попить, поспасть, поесть, по...",14
3,Спасибо. 🙏,Q6xX28WaNRc,[],0
4,Моё уважение! Всё по уму! Я в деле! Ни травлюс...,Q6xX28WaNRc,"[уважение, ум, дело, травиться, яд, активно, с...",23


In [ ]:
yt_df.shape

(278781, 4)

In [ ]:
yt_df.to_csv('full_yt_df2.csv')

In [ ]:
yt_df.head()

,raw_text,videoId,text,length
0,Пичкать маленького ребенка фтором🤦‍♂️,ThtuIQRnTN4,"[пичкать, маленький, ребёнок]",3
1,Какие люди тупые бывают...,ThtuIQRnTN4,"[тупой, бывать]",2
2,"Первый раз вижу, что человек оправдывается за ...",ThtuIQRnTN4,"[видеть, оправдываться, удовлетворение, базовы...",5
3,А где это сдают все на переработку? У нас даже...,ThtuIQRnTN4,"[сдавать, переработка, макулатура, частный, ли...",10
4,"Мне не понравилось в этом видео, что Вы сидите...",ThtuIQRnTN4,"[понравиться, видео, сидеть, экран, поза]",5
